# CIS 545 Homework 2: SQL
## Due: Monday, February 21, 2022 by 10pm 
### Worth 100 points in total

Welcome to Homework 2! By now, you should be familiar with the world of data science and the Pandas library. This assignment will focus on broadening both of these horizons by covering hierarchical data, graphs, and traversing relationships as well as a new tool to add to your data science arsenal: SQL.

Through this homework, we will familiarize ourselves with SQL (specifically **pandasql**) and explore a dataset that involves movie statistics and reviews courtesy of Rotten Tomatoes. We will finish off the homework with some text analysis.

We are introducing a lot of new things in this homework, and it is often where students start to get lost in the data science sauce, so we **strongly** encourage you to review the slides/material as you work through this assignment and will try to link the most relevant sections!

**Before you Begin**
- Be sure to click "Copy to Drive" to make sure you are working on your own personal version of the homework
- Read the Piazza and FAQ for updates! If you have been stuck, chances are other students are too! We don't want you to waste away for two hours trying to get that last point on the autograder so do check Piazza for similar struggles or even homework bugs that will be clarified in the FAQ :) 

## Part 0: Libraries and Set Up Jargon (The usual wall of imports)

In [1]:
!pip3 install penngrader
!pip install pandasql

In [2]:
!pip install pandas==1.1.5

In [3]:
from penngrader.grader import *
import pandas as pd
import datetime as dt
import re
import pandasql as ps #SQL on Pandas Dataframe
import nltk
nltk.download('punkt')

from wordcloud import WordCloud
import matplotlib.pyplot as plt 
from collections import Counter
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Three datasets we're using
! wget -nc https://storage.googleapis.com/penn-cis545/rotten_tomatoes_movies.csv
! wget -nc https://storage.googleapis.com/penn-cis545/rotten_tomatoes_critic_reviews.csv
! wget -nc https://storage.googleapis.com/penn-cis545/MoviesOnStreamingPlatforms.csv

File ‘rotten_tomatoes_movies.csv’ already there; not retrieving.

File ‘rotten_tomatoes_critic_reviews.csv’ already there; not retrieving.

File ‘MoviesOnStreamingPlatforms.csv’ already there; not retrieving.



In [5]:
print(pd.__version__)
# Make sure it's 1.1.5

1.1.5


### PennGrader Setup

In [6]:
# PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW 
# TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 83510311
# YOUR PENN-ID GOES HERE AS AN INTEGER #

In [7]:
grader = PennGrader(homework_id = 'CIS545_Spring_2022_HW2', student_id = STUDENT_ID)

PennGrader initialized with Student ID: 83510311

Make sure this correct or we will not be able to store your grade




# Section 1: Welcome to the movies!

<br>
<center><img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/6/6f/Rotten_Tomatoes_logo.svg/2880px-Rotten_Tomatoes_logo.svg.png" width= "500" align ="center"/></center>
<br>

I'm sure everyone has looked on the internet at some point in their lives for reviews, whether it be for products, locations, or services. Reviews are important because they give us knowledge about how good (or bad) the reviewed item was, at least in the past. Rotten Tomatoes contains one of the most comprehensive databases of movie/TV show reviews. Its name comes from how audience members in the past tended to throw rotten tomatoes when they disliked a stage performance. Formed in 1998 by a bunch of students, it has withstood the test of time, and the rest is history... that you can read about yourself on [Wikipedia](https://en.wikipedia.org/wiki/Rotten_Tomatoes). 


In this homework, we'll be exploring some data about movies including:

*   Movies: data about movies in the Rotten Tomatoes database.

*   Reviews: data about each review that was posted on Rotten Tomatoes.

*   Streamed movies: data about the movies that are currently available on major streaming platforms. 


We'll be parsing this data into dataframes and relations, and then exploring how to query and assemble the tables into results. We will primarily be using PandaSQL, but for some of the initial questions, we will ask you to perform the same operations in Pandas as well, so as to familiarize you with the differences and similarities of the two. `

## Part 1: Load & Process our Datasets [9 points total]

Before we get into the data, we first need to load and clean our datasets. 

**TODO**:
* Load and save the `rotten_tomatoes_movies.csv` to a dataframe called `movies_df`.
* Load and save the `rotten_tomatoes_critic_reviews.csv` to a dataframe called `reviews_df`.
* Load and save the `MoviesOnStreamingPlatforms.csv` to a dataframe called `streaming_df` without the index column being included.

In [8]:
# TODO: Import the datasets to pandas dataframes -- make sure the dataframes are named correctly! 
movies_df = pd.read_csv('rotten_tomatoes_movies.csv')

In [9]:
# view movies_df to make sure the import was successful
reviews_df = pd.read_csv('rotten_tomatoes_critic_reviews.csv')

In [10]:
# view reviews_df to make sure the import was successful
streaming_df = pd.read_csv('MoviesOnStreamingPlatforms.csv')

In [11]:
# view streaming_df to make sure the import was successful
streaming_df.head()

,Unnamed: 0,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type
0,0,1,The Irishman,2019,18+,98/100,1,0,0,0,0
1,1,2,Dangal,2016,7+,97/100,1,0,0,0,0
2,2,3,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,0
3,3,4,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0,0
4,4,5,Roma,2018,18+,94/100,1,0,0,0,0


### 1.1 Data Preprocessing

Next, we are going to want to clean up our dataframes, namely `movies_df` and `reviews_df`, by 1) fixing column names, 2) changing datatypes, 3) cleaning text, and 4) handling nulls.

First, let us view the first few rows of `movies_df`. You may also call `.info()` to view the specifics of the dataframe. This is a good first step to take for Exploratory Data Analysis (EDA).

In [12]:
movies_df.head(n=1)

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,streaming_release_date,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,2015-11-25,119.0,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,73,76


In [13]:
# view info information regarding movies_df
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17712 entries, 0 to 17711
Data columns (total 21 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   rotten_tomatoes_link              17712 non-null  object 
 1   movie_title                       17712 non-null  object 
 2   movie_info                        17391 non-null  object 
 3   critics_consensus                 9134 non-null   object 
 4   content_rating                    17712 non-null  object 
 5   genres                            17693 non-null  object 
 6   directors                         17518 non-null  object 
 7   authors                           16170 non-null  object 
 8   actors                            17360 non-null  object 
 9   original_release_date             16546 non-null  object 
 10  streaming_release_date            17328 non-null  object 
 11  runtime                           17398 non-null  float64
 12  prod

#### 1.1.1 Cleaning `movies_df`

`.info()` gives us meaningful information regarding columns, their types, and the amount of nulls, based on which we can now clean our dataframe. 

**TODO**:
* Drop the columns `critics_consensus` and `production_company`.
* Replace all nulls in the column `genres` with the string `"No Genre"`.
* Drop all rows that have at least one null.
* Cast columns `original_release_date`,`streaming_release_date` to be type `datetime64[ns]`.
* Cast columns `runtime`,`tomatometer_count`, `tomatometer_rating`, `audience_count`, and `audience_rating` to be type `int64`.

In [14]:
# TODO: clean movies_df
movies_df = movies_df.drop(columns = ['critics_consensus', 'production_company'])
movies_df['genres'] = movies_df['genres'].fillna("No Genre")
movies_df = movies_df.dropna(how = 'any')
movies_df['original_release_date'] = movies_df['original_release_date'].apply(pd.to_datetime)
movies_df['streaming_release_date'] = movies_df['streaming_release_date'].apply(pd.to_datetime)
movies_df[['runtime', 'tomatometer_count', 'tomatometer_rating', 'audience_count', 'audience_rating']] = movies_df[['runtime', 'tomatometer_count', 'tomatometer_rating', 'audience_count', 'audience_rating']].astype('int64')

In [15]:
# 3 points
grader.grade(test_case_id = 'test_cleaning_movies', answer = movies_df.head(1000))

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 1.1.2 Processing Genres

`movies_df` also contains a `genres` column that represents an exhaustive list of all the genres a particular movie falls under. The comma-separated string format this column is in isn't too useful to us at the moment...

**TODO**:
- Copy over `movies_df` into a new dataframe called `exploded_movies_df`. 
- Split the genres listed such that each row contains only one listed genre (if a particular movie has 2 genres, that row will appear twice for each of the genres) - call this column `genre`
- Strip the `genre` column of any leading or trailing whitespaces


**Hint**: See the `.explode()` and `.strip()` functions

In [16]:
# TODO: create new dataframe and genre column
exploded_movies_df = movies_df.copy()
exploded_movies_df['genres'] = exploded_movies_df['genres'].str.split(pat=',', expand=False)
exploded_movies_df = exploded_movies_df.explode('genres')
exploded_movies_df['genres'] = exploded_movies_df['genres'].str.strip()
exploded_movies_df = exploded_movies_df.rename(columns={"genres":"genre"})

In [17]:
# 3 points
grader.grade(test_case_id = 'test_genre_processing', answer = exploded_movies_df.head(1000))

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 1.1.3 Cleaning `reviews_df`

Then, let's take a look at cleaning `reviews_df`.

In [18]:
reviews_df.head(n=1)

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...


In [19]:
#view info of reviews_df
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130017 entries, 0 to 1130016
Data columns (total 8 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   rotten_tomatoes_link  1130017 non-null  object
 1   critic_name           1111488 non-null  object
 2   top_critic            1130017 non-null  bool  
 3   publisher_name        1130017 non-null  object
 4   review_type           1130017 non-null  object
 5   review_score          824081 non-null   object
 6   review_date           1130017 non-null  object
 7   review_content        1064211 non-null  object
dtypes: bool(1), object(7)
memory usage: 61.4+ MB



**TODO**:
* Drop all rows that have a null value in the column `review_score` or `review_content`.
* Replace each null in the `critic_name` column with the string `"Anonymous"`.
* Convert column `review_date` into type `datetime64[ns]`.


In [20]:
#TODO: Clean reviews_df
reviews_df = reviews_df.dropna(subset=['review_score', 'review_content'], how = 'any')
reviews_df['critic_name'] = reviews_df['critic_name'].fillna("Anonymous")
reviews_df['review_date'] = reviews_df['review_date'].apply(pd.to_datetime)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
# 3 points
grader.grade(test_case_id = 'test_cleaning_reviews', answer = reviews_df.head(1000))

Correct! You earned 3/3 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 1.2 Your Sandbox 

.info() is just one of many basic tools that you can use for Exploratory Data Analysis (EDA). Instead of throwing you straight into the deep end, we wanted to give you a chance to take some time and explore the data on your own. **This section is not graded**, so for the speedrunners out there feel free to just jump in, but we wanted to at least give you a small space to utilize your EDA toolkit to familiarize yourself with all the data you just downloaded.

Some suggestions to get you started:
- `df.head()`
- `df.describe()`
- `Series.unique()`

In [22]:
# Your EDA here! Feel free to add more cells
exploded_movies_df.describe()

,runtime,tomatometer_rating,tomatometer_count,audience_rating,audience_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
count,33005.000000,33005.000000,33005.000000,33005.000000,3.300500e+04,33005.000000,33005.000000
mean,103.663869,60.624663,59.350432,60.946069,1.714511e+05,37.541585,21.821027
std,19.139535,28.073298,68.417631,20.231864,1.916547e+06,52.484828,30.874536
min,8.000000,0.000000,5.000000,0.000000,5.000000e+00,0.000000,0.000000
25%,91.000000,38.000000,13.000000,45.000000,1.268000e+03,6.000000,3.000000
50%,100.000000,66.000000,31.000000,63.000000,6.091000e+03,18.000000,9.000000
75%,112.000000,85.000000,80.000000,78.000000,3.629600e+04,46.000000,26.000000
max,266.000000,100.000000,535.000000,100.000000,3.579764e+07,497.000000,303.000000


## Part 2: Exploring the Data with PandasSQL (and Pandas) [75 points total]

Now that you are familiar (or still unfamiliar) with the dataset, we will now introduce you to SQL, or more specifically **pandasql**: a package created to allow users to query pandas DataFrames with SQL statements.

The typical flow to use pandasql (shortened to **ps**) is as follows:
1. Write a SQL query in the form of a string (Tip: use triple quotes """x""" to write multi-line strings)
2. Run the query using **ps.sqldf(your_query, locals())**

Pandasql is convenient in that it allows you to reference the dataframes that are currently defined in your notebook, so you will be able to fully utilize the dataframes `movies_df`, `reviews_df` and `streaming_df` that you have created above!

Given that it is a brand new language, we wanted to give you a chance to directly compare the similarities/differences of the pandas that you already know and the SQL that you are about to learn. Thus, for each of the simpler queries, we ask that you **look into the question twice: once with pandas and once with pandasql**. 

Each answer will thus require both a `pd_` and `sql_` prefixed-dataframe that you will submit seperately to the autograder. **We will be reviewing your code to make sure you wrote the code in the corresponding languages.**


###2.1 Movie Recommendations

#### 2.1.1 What movies have good reviews from critics and audience?

`movies_df` contains all sorts of movies. We all love good movies, so let's try to separate the good from the bad.

Rotten Tomatoes offers two major aggregate ratings: 
- one by critics, in the column `tomatometer_status`, and has three possible values: `Certified-Fresh`,`Fresh`, and `Rotten`.
- the other by audience members, in the column `audience_status`, and has two possible values: `Upright` (good) and `Spilled` (bad).

**TODO:** Using **pandas**, filter out movies from `movies_df` that are `Certified-Fresh` or `Fresh` into a new dataframe named `good_critics_df`, which should have the following schema:

>rotten_tomatoes_link | movie_title
>--- | ---



In [23]:
# TODO: Use pandas to obtain good_critics_df
good_critics_df = movies_df[movies_df['tomatometer_status'] != "Rotten"][['rotten_tomatoes_link', 'movie_title']]

In [24]:
# 2 points
grader.grade(test_case_id = 'test_good_critics_df', answer = good_critics_df)

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


then, let's consider the ratings by movie-goers.

**TODO:** Using **pandasql**, filter out movies from `movies_df` that are rated as `Upright` by the audience, into a new dataframe named `good_audience_df`, which should have the following schema:

>rotten_tomatoes_link | movie_title
>--- | ---

In [25]:
# TODO: Use pandasql to obtain good_audience_df
good_audience_query = """
SELECT rotten_tomatoes_link, movie_title 
FROM movies_df 
WHERE audience_status = "Upright"
"""

good_audience_df = ps.sqldf(good_audience_query, locals())

In [26]:
# 2 points
grader.grade(test_case_id = 'test_good_audience_df', answer = good_audience_df)

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


Now, we can simply join these dataframes to get a table of all the best movies and binge watch one every night! But wait! Sometimes, the opinions of critics and audience may differ, so we need to make sure that the good movies we watch are ones that are viewed positively by both critics and audiences. 

**TODO**: Using **pandas and pandasql**, filter out movies with **mixed reviews** (ie. audience and critic opinions differ) to a new dataframe `pd/sql_mixed_movies_df`, ordered by lexicographic order of `movie_title`.

For this question, you should **NOT** use `movies_df` or `exploded_movies_df`.

Some tips:
* For pandas, explore the `indicator` parameter within the `merge()` function.
* For pandasql, considering using `EXCEPT`.

In [27]:
# TODO: Filter out movies with mixed reviews using pandas
pd_mixed_movies_df = pd.merge(good_audience_df, good_critics_df, on=['movie_title','rotten_tomatoes_link'], how='outer', indicator=True)
pd_mixed_movies_df = pd_mixed_movies_df[pd_mixed_movies_df['_merge'] != "both"].sort_values(by='movie_title')
pd_mixed_movies_df = pd_mixed_movies_df.drop(columns=['_merge'])

In [28]:
# 4 points
grader.grade(test_case_id = 'test_pd_mixed_movies', answer = pd_mixed_movies_df)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [29]:
# TODO: Filter out movies with mixed reviews from good_critics_df and good_audience_df
mixed_query = """
SELECT * FROM
  (
    SELECT * FROM good_critics_df
    EXCEPT 
    SELECT * FROM good_audience_df
  )
UNION
SELECT * FROM
  (
    SELECT * FROM good_audience_df 
    EXCEPT
    SELECT * FROM good_critics_df
  )
  ORDER BY movie_title;
"""

sql_mixed_movies_df = ps.sqldf(mixed_query, locals())

In [30]:
# 4 points
grader.grade(test_case_id = 'test_sql_mixed_movies', answer = (mixed_query,sql_mixed_movies_df))

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


**TODO:**

Using **pandas/pandasql** and `pd/sql_mixed_movies_df`, find the top 10 mixed movies with the **largest absolute difference** in critic review score and audience review score. You should calculate this as the absolute difference between `tomatometer_rating` and `audience_rating`, and store this in a column named `diff`. If multiple movies have the same `diff` value, order their rows in lexicographic order.

Format the output as a dataframe called `pd/sql_top_10_mixed_movies_df` that has the following schema:

>movie_title | diff
>--- | ---


In [31]:
# TODO: pandas
pd_top_10_mixed_movies_df = pd.merge(pd_mixed_movies_df, movies_df[['movie_title','rotten_tomatoes_link','tomatometer_rating', 'audience_rating']], 
                                  on=['movie_title','rotten_tomatoes_link'], how='left')
pd_top_10_mixed_movies_df['diff'] = (pd_top_10_mixed_movies_df.tomatometer_rating - pd_top_10_mixed_movies_df.audience_rating).abs()
pd_top_10_mixed_movies_df = pd_top_10_mixed_movies_df.sort_values(by=['diff','movie_title'], ascending=[False, True])[0:10]
pd_top_10_mixed_movies_df = pd_top_10_mixed_movies_df.drop(columns=['rotten_tomatoes_link','tomatometer_rating', 'audience_rating'])
pd_top_10_mixed_movies_df

,movie_title,diff
46,96 Souls,95
1151,Hating Breitbart,92
1353,Is That a Gun in Your Pocket?,92
2527,The After Party,87
1959,Outside the Law,86
878,Fall,84
3357,Vengeance Valley,82
2975,The Music of Silence,81
166,All Relative,80
2368,Sparkle,80


In [32]:
#TODO: pandasql
top_10_mixed_query = """
SELECT s.movie_title, ABS(tomatometer_rating - audience_rating) AS diff
FROM sql_mixed_movies_df s
LEFT JOIN movies_df m
ON s.movie_title = m.movie_title
AND s.rotten_tomatoes_link = m.rotten_tomatoes_link
ORDER BY diff DESC, s.movie_title ASC
LIMIT 10
"""

sql_top_10_mixed_movies_df = ps.sqldf(top_10_mixed_query, locals())
sql_top_10_mixed_movies_df

,movie_title,diff
0,96 Souls,95
1,Hating Breitbart,92
2,Is That a Gun in Your Pocket?,92
3,The After Party,87
4,Outside the Law,86
5,Fall,84
6,Vengeance Valley,82
7,The Music of Silence,81
8,All Relative,80
9,Sparkle,80


In [33]:
# 6 points
grader.grade(test_case_id = 'test_top_10_mixed', answer = (top_10_mixed_query,pd_top_10_mixed_movies_df,sql_top_10_mixed_movies_df))

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


#### 2.1.2 What movies can we recommend to Professor Davidson?

Professor Davidson wants to take a break from teaching CIS 545. She is considering watching a movie on her Netflix account, and is gathering movie recommendations from you.

Because her favourite holiday is halloween, Professor Davidson is looking to watch a horror movie. But first, she wants to answer a long-standing question in her mind: are there more horror movies during October? 

**TODO**: Using **pandas and pandasql**,
- add a column `month` to movies_df that contains the month of the movie's original date of release.
- Find the number of movies classified as `Horror`, grouped by month. Store this number in a column `num_movies` in descending order, and store the results in `pd/sql_horror_df` which should have the following format:

>month | num_movies
>--- | ---


In [34]:
# TODO: pandas version
movies_df['month'] = movies_df['original_release_date'].dt.month
pd_horror_df = movies_df.copy()
pd_horror_df = pd_horror_df[pd_horror_df['genres'].str.contains('Horror')]
pd_horror_df = pd_horror_df.groupby('month').aggregate({'movie_title': 'count'}).reset_index()
pd_horror_df = pd_horror_df.rename(columns={'movie_title':'num_movies'}).sort_values('num_movies', ascending = False)


In [35]:
# TODO: pandasql version
horror_query = """
SELECT CAST(strftime('%m',original_release_date) as integer) month, COUNT(*) num_movies
FROM movies_df
WHERE genres LIKE '%Horror%'
GROUP BY month
ORDER BY num_movies DESC
"""

sql_horror_df = ps.sqldf(horror_query, locals())

In [36]:
# 4 points
grader.grade(test_case_id = 'test_horror_month', answer = (horror_query, pd_horror_df, sql_horror_df))

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


It seems like October is not the most popular month in which horror movies are released. 

**TODO:**
Using **pandas and pandasql**, find the movies classified as `Horror` that were originally released in **January**. Store the results in `pd/sql_jan_df` which should have the following format, sorted in descending order of `audience_count`:

>rotten_tomatoes_link | movie_title | audience_count
>--- | --- | ---

In [37]:
# TODO: pandas version
pd_jan_df = movies_df.copy()
pd_jan_df['month'] = pd_jan_df['original_release_date'].dt.month
pd_jan_df = pd_jan_df[pd_jan_df['genres'].str.contains('Horror')]
pd_jan_df = pd_jan_df[pd_jan_df['month'] == 1]
pd_jan_df = pd_jan_df[['rotten_tomatoes_link',	'movie_title',	'audience_count']].sort_values('audience_count', ascending=False)

In [38]:
# TODO: pandasql version
jan_query = """
SELECT rotten_tomatoes_link,	movie_title,	audience_count
FROM movies_df
WHERE genres LIKE '%Horror%'
AND CAST(strftime('%m',original_release_date) as integer) = 1
ORDER BY audience_count DESC
"""

sql_jan_df = ps.sqldf(jan_query, locals())

In [39]:
# 4 points
grader.grade(test_case_id = 'test_horror_jan', answer = (jan_query, pd_jan_df, sql_jan_df))

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


With all the information about these movies, we are ready to help Professor Davidson find her dream movie. In particular, she wants a movie that: 
- is a horror movie
- was originally released in January 
- has been watched by at least 50000 audience members, with an overall `audience_rating` of `Upright`.  

**TODO**:
* Using the information provided above, and using **pandasql**, find the top 10 movies that fit her criteria, sorted by descending order of `audience_count`. Store this in a dataframe called `movie_recs_df`, that has the following schema:

>rotten_tomatoes_link | movie_title | audience_count
>--- | --- | ---

**Hint**: you may find it helpful to leverage `good_audience_df` and `pd_jan_df`.

In [40]:
# TODO: use pandasql to find top 10 movies that fit criteria
recs_query = """
SELECT b.rotten_tomatoes_link,	b.movie_title,	b.audience_count
FROM
  (
    SELECT rotten_tomatoes_link,	movie_title
    FROM good_audience_df
    INTERSECT
    SELECT rotten_tomatoes_link,	movie_title
    FROM pd_jan_df
  ) a
LEFT JOIN pd_jan_df b
ON a.movie_title = b.movie_title
WHERE b.audience_count >= 50000
ORDER BY b.audience_count DESC
LIMIT 10
"""

movie_recs_df = ps.sqldf(recs_query, locals())

In [41]:
# 6 points
grader.grade(test_case_id = 'test_recs', answer = (recs_query, movie_recs_df))

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.2 Good Critic Reviews 

Now let us switch gears and examine `reviews_df`. In particular, we want to find the critics who are good at writing reviews.

The first criteria that defines a good reviewer is balance and objectivity. Just like people (other than me), no movie is perfect, and so we probably want reviews to cover both the good and the bad. 

**TODO**: Using `reviews_df` and **pandasql**, find the reviews that contain both the word `"good"` and the word `"bad"` (case-insensitive). Include substrings (ie. it can be part of a word, such as '**good**-natured' or 'for**bad**e').

Store the results in `balanced_df`, ordered by lexicographical order of `review_content`, with the following schema:

>critic_name | review_content | 
>--- | --- | 

HINT: This involves checking for string similarity.
- for Pandas, you can use str.contains()
- for Pandasql, you may take a look at `LIKE()` and the wildcards `_` and/or `%`.



In [42]:
# TODO: use pandasql to find reviews with both words "good" and "bad"
balanced_query = """
SELECT critic_name,	review_content
FROM reviews_df
WHERE review_content LIKE ('%good%')
AND review_content LIKE ('%bad%')
ORDER BY review_content
"""
balanced_df = ps.sqldf(balanced_query, locals())

In [43]:
# 2 points
grader.grade(test_case_id = 'test_balanced_df', answer = (balanced_df, balanced_query))

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


*Good cause, bad movie.* One of my favourite quotes.

The second criteria we have for good critics is tenure. Let us find the "seasoned" reviewers, ie. those who have been writing reviews for a long period of time.

**TODO**: Using `reviews_df` and **pandasql**:
- Create a column `date_diff` which contains the difference (in number of days) between each critic's first review and most recent review
- Only include reviews from the 21st Century (ie. published from 2000 onward)
- Only include critics whose first and most recent reviews are at least 10 years **(3652 days)** apart
- Do not include reviews by `'Anonymous'`
- Make sure your `date_diff` column contains integer values (HINT: see `CAST()`).

You may consider casting the dates into a [format](https://www.sqlite.org/lang_datefunc.html) that will help produce the difference in days. 

Store the results in `critics_time_df`, ordered by descending order of `date_diff`, with the following schema:

>critic_name | date_diff | 
>--- | --- | 

In [44]:
reviews_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 758709 entries, 3 to 1130016
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   rotten_tomatoes_link  758709 non-null  object        
 1   critic_name           758709 non-null  object        
 2   top_critic            758709 non-null  bool          
 3   publisher_name        758709 non-null  object        
 4   review_type           758709 non-null  object        
 5   review_score          758709 non-null  object        
 6   review_date           758709 non-null  datetime64[ns]
 7   review_content        758709 non-null  object        
dtypes: bool(1), datetime64[ns](1), object(6)
memory usage: 47.0+ MB


In [45]:
# TODO: use pandasql to find seasoned reviewers
time_query = """
SELECT critic_name, CAST(MAX(JULIANDAY(review_date))-MIN(JULIANDAY(review_date)) as integer) date_diff
FROM reviews_df
WHERE JULIANDAY(review_date) >= 2451544.5
AND critic_name <> "Anonymous"
GROUP BY critic_name
HAVING date_diff >= 3652
ORDER BY date_diff DESC
"""

critics_time_df = ps.sqldf(time_query, locals())

In [46]:
# 6 points
grader.grade(test_case_id = 'test_critic_review_time', answer = (critics_time_df, time_query))

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.3 What do we watch on Netflix tonight?

Now let's run a tougher analysis. I'm sure we've all experienced the aimless scrolling through the Netflix homepage, trying (and sometimes failing) to find a movie that interests us. It's painful to manually search up the reviews of each movie we see, so why don't we try to automate this process? Let's determine the best on Netflix for each genre using both critics' and audiences' opinions on Rotten Tomatoes.

This (and the next 2 questions) will require you to write a [nested SQL query](https://learnsql.com/blog/sql-nested-select/). That is, there will be at least one SELECT statement inside of another SELECT statement. This means that you should **NOT** write two separate SQL commands and call ps.sqldf() twice. 

**TODO**: Using `exploded_movies_df` and `streaming_df`:
- Find all movies on Netflix, then **for each genre,** identify the ones with the highest combined RT tomatometer and audience rating (include all ties). Call this new column `combined_rt_rating`.
- Rename the key `genre_category`, and sort your final result by this column in ascending order.

Though it could be helpful, **you do NOT have to implement this in pandas**. Store the results in `netflix_best_df`, which should have the following format:

>movie_title | genres | genre_category | combined_rt_rating
>--- | --- | --- | ---

where `genres` represents all the genres of the movie, and `genre_category` represents the genre that the movie's RT ratings are compared to.

In [47]:
streaming_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9515 entries, 0 to 9514
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       9515 non-null   int64 
 1   ID               9515 non-null   int64 
 2   Title            9515 non-null   object
 3   Year             9515 non-null   int64 
 4   Age              5338 non-null   object
 5   Rotten Tomatoes  9508 non-null   object
 6   Netflix          9515 non-null   int64 
 7   Hulu             9515 non-null   int64 
 8   Prime Video      9515 non-null   int64 
 9   Disney+          9515 non-null   int64 
 10  Type             9515 non-null   int64 
dtypes: int64(8), object(3)
memory usage: 817.8+ KB


In [48]:
# TODO: use pandasql to find the best on Netflix

netflix_best_query = """
WITH title_best AS (
  SELECT genre, MAX(tomatometer_rating + audience_rating) combined_rt_rating
  FROM exploded_movies_df e
  INNER JOIN streaming_df s
  ON e.movie_title = s.Title
  WHERE s.Netflix = 1
  GROUP BY genre
),
title_comb AS (
  SELECT movie_title, (tomatometer_rating + audience_rating) combined_rt_rating, genre, rotten_tomatoes_link
  FROM exploded_movies_df ex
  INNER JOIN streaming_df st
  ON ex.movie_title = st.Title
  WHERE st.Netflix = 1
)

SELECT tc.movie_title, m.genres, tb.genre genre_category, tb.combined_rt_rating
FROM title_best tb
INNER JOIN title_comb tc
ON tc.genre = tb.genre
AND tc.combined_rt_rating = tb.combined_rt_rating
INNER JOIN movies_df m
ON m.movie_title = tc.movie_title
AND m.rotten_tomatoes_link = tc.rotten_tomatoes_link
ORDER BY genre_category
"""

netflix_best_df = ps.sqldf(netflix_best_query, locals())

In [49]:
# 10 points
grader.grade(test_case_id = 'test_netflix_best', answer = (netflix_best_query, netflix_best_df))

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.4 Which months of the year does each streaming service have the most number of movies available?

Maybe you don't care about picking out the "good" movies from the "bad" ones. Maybe you just want to watch as many movies as possible each month. Still, paying those monthly bills for all these streaming services (which seem to be increasing in number every day) doesn't seem to be a sustainable strategy. It would be much easier to pay for certain streaming services in the month they have the most movies.

**TODO**: Using `movies_df` and `streaming_df`,
- Count the number of movies for each streaming service per month using the `month` column in `movies_df` you created earlier.
- For each streaming service, find the month in which they have the most movies available and how many movies are available in that month. Include ties, if there are any.

Store the results in `best_streaming_df`, which should have the following format:

>streaming_service | max_month | num_movies
>--- | --- | --- |

For example, if Netflix has the most movies (100) in October and Hulu has the most movies (90) in January, your query should output:

streaming_service  | max_month        | num_movies
-------------------|------------------|--------------
Netflix    | 10                       | 100
Hulu       | 1                        | 90

and so on, for all four streaming services.

**Hints:**
- Make use of `UNION ALL` to stack tables with the same column names.

In [50]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14627 entries, 0 to 17711
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   rotten_tomatoes_link              14627 non-null  object        
 1   movie_title                       14627 non-null  object        
 2   movie_info                        14627 non-null  object        
 3   content_rating                    14627 non-null  object        
 4   genres                            14627 non-null  object        
 5   directors                         14627 non-null  object        
 6   authors                           14627 non-null  object        
 7   actors                            14627 non-null  object        
 8   original_release_date             14627 non-null  datetime64[ns]
 9   streaming_release_date            14627 non-null  datetime64[ns]
 10  runtime                           14627 non-nu

In [51]:
# TODO: use pandasql to find the month where each streaming service has the most movies
best_streaming_query = """
WITH list AS (
  SELECT 'Netflix' streaming_service, COUNT(*) num_movies, month
  FROM streaming_df 
  INNER JOIN movies_df 
  ON Title = movie_title
  WHERE Netflix = 1
  GROUP BY month

  UNION
  SELECT 'Hulu' streaming_service, COUNT(*) num_movies, month
  FROM streaming_df 
  INNER JOIN movies_df 
  ON Title = movie_title
  WHERE Hulu = 1
  GROUP BY month

  UNION
  SELECT 'Prime Video' streaming_service, COUNT(*) num_movies, month
  FROM streaming_df 
  INNER JOIN movies_df 
  ON Title = movie_title
  WHERE `Prime Video` = 1
  GROUP BY month
  
  UNION
  SELECT 'Disney+' streaming_service, COUNT(*) num_movies, month
  FROM streaming_df 
  INNER JOIN movies_df 
  ON Title = movie_title
  WHERE `Disney+` = 1
  GROUP BY month
),
most_list AS(
  SELECT streaming_service, MAX(num_movies) num_movies
  FROM list
  GROUP BY streaming_service
)
SELECT m.streaming_service, l.month max_month, m.num_movies
FROM most_list m
INNER JOIN list l
ON m.streaming_service = l.streaming_service
AND m.num_movies = l.num_movies
"""

best_streaming_df = ps.sqldf(best_streaming_query, locals())
best_streaming_df

,streaming_service,max_month,num_movies
0,Disney+,6,44
1,Hulu,8,65
2,Netflix,10,92
3,Prime Video,9,126


In [52]:
# 10 points
grader.grade(test_case_id = 'test_best_streaming', answer = (best_streaming_query, best_streaming_df))

Correct! You earned 10/10 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.5 What movies are underrated by critics (ft. top critics)?

[According to the Rotten Tomatoes website](https://www.rottentomatoes.com/critics/top_critics), "Top Critic is a designation created to distinguish Tomatometer-approved critics who excel at their craft. Critics selected are well-established, influential, and prolific..." Although all critics on RT are officially approved, we want to establish a top-critic-only score that only includes the opinions of the "cream of the crop". Certain films could be poorly received by regular critics, but top critics see something special in them. Let's try and find these underappreciated movies!

**TODO**: Using `reviews_df` and `movies_df`:
- Filter out all non-top critics, then compute a new Tomatometer rating for each movie based on its proportion of Fresh reviews to total reviews (*both statistics among top critics only*). Call this new column `top_critic_tomatometer`, and **round this percentage to the nearest integer.**
- Constrain the movie pool to **only ones that have more than 5 top critic reviews** to mitigate outlier impact.
- Retrieve the top 10 movies with the highest difference between `top_critic_tomatometer` and its provided Tomatometer rating (`tomatometer_rating`) - name this difference `score_diff`. 
  - `top_critic_tomatometer` - `tomatometer_rating`. Not the absolute value of the difference.

Store the results in `underrated_movies_df`, which should have the following format:

>movie_title | top_critic_tomatometer | tomatometer_rating | score_diff
>--- | --- | --- | ---

**Hints:**
- SQLite stores booleans as integers 1 (True) and 0 (False), respectively.
- Tomatometer ratings are provided as percentages (i.e. 84 not 0.84), so make sure your top critic scores are as well.
- Since there are numerous remakes included in the dataset with the same name as the original/multiple other versions, make sure you conduct aggregate/join operations on `rotten_tomatoes_link` to ensure uniqueness.
- Dividing two integers in SQL yields a rounded integer.

In [53]:
reviews_df.head()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
6,m/0814255,Nick Schager,False,Slant Magazine,Rotten,1/4,2010-02-10,Harry Potter knockoffs don't come more transpa...
7,m/0814255,Bill Goodykoontz,True,Arizona Republic,Fresh,3.5/5,2010-02-10,"Percy Jackson isn't a great movie, but it's a ..."
8,m/0814255,Jordan Hoffman,False,UGO,Fresh,B,2010-02-10,"Fun, brisk and imaginative"
9,m/0814255,Jim Schembri,True,The Age (Australia),Fresh,3/5,2010-02-10,"Crammed with dragons, set-destroying fights an..."


In [102]:
# TODO: use pandasql to find the regular critics' underrated movies
underrated_movies_query = """
WITH top AS(
  SELECT rotten_tomatoes_link, CASE WHEN review_type='Fresh' THEN 1 ELSE 0 END counter
  FROM reviews_df
  WHERE top_critic = 1
),
top_per AS(
  SELECT rotten_tomatoes_link, 100*SUM(counter)/COUNT(*) top_critic_tomatometer
  FROM top
  WHERE rotten_tomatoes_link IN (SELECT DISTINCT rotten_tomatoes_link FROM reviews_df WHERE review_type = 'Fresh')
  GROUP BY rotten_tomatoes_link
  HAVING COUNT(*)>5
)

SELECT b.movie_title, a.top_critic_tomatometer, b.tomatometer_rating, (a.top_critic_tomatometer - b.tomatometer_rating) score_diff
FROM top_per a
INNER JOIN movies_df b
ON a.rotten_tomatoes_link = b.rotten_tomatoes_link
ORDER BY score_diff DESC
LIMIT 10
"""

**Note:** This query should take no more than a minute to run. If yours has been running for longer than a few minutes, then you're probably on the wrong track.

In [103]:
underrated_movies_df = ps.sqldf(underrated_movies_query, locals())

In [104]:
# 15 points
grader.grade(test_case_id = 'test_underrated_movies', answer = (underrated_movies_query, underrated_movies_df))

Correct! You earned 15/15 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Part 3: Working with Text Data [16 points]

Shifting gears, let's now try to do some text-based analysis. Text data is complex, but can also be used to generate extremely interpretable results, making it valuable and interesting. 

Throughout this section, we will attempt to answer the following question:

**How do reviews by top critics differ from reviews made by non-top critics?**


###3.1 Extract Data
**TODO**: Using `reviews_df`, create two dataframes:
* `top_critics_df`: where `top_critic` is True
* `regular_critics_df`: where `top_critic` is False.

Then, for each dataframe, convert the `review_content` column into two lists called `top_content` and `regular_content`, respectively.

In [111]:
# TODO: create two dataframes & two lists
top_critics_df = reviews_df[reviews_df['top_critic'] == True]
top_content = top_critics_df['review_content']
regular_critics_df= reviews_df[reviews_df['top_critic'] == False]
regular_content = regular_critics_df['review_content']

In [112]:
# 1 point
grader.grade(test_case_id = 'test_top_content_df', answer = len(top_content))

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [113]:
# 1 point
grader.grade(test_case_id = 'test_regular_content_df', answer = len(regular_content))

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


###3.2 Tokenize the Text

Here, we are going to split up the content into a list of words. We will use the **nltk** package, which contains an extensive set of tools to process text. Of course, like regex, this homework would be miles long if we really went into detail, so we are only going to utilize the following components:
- `nltk.word_tokenize()`: a function used to tokenize our text
- `nltk.corpus.stopwords`: a list of commonly used words such as "a","an","in" that are often ignored in text-related analysis

Note that prior to this step for text analysis, we would typically clean the text first using regex. We didn't have to do that because the dataset was already well-cleaned, bt keep that in mind.

**TODO:** First, use **stopwords** to create a set of the most common english stopwords. Then, implement **tokenized_content(content)** that takes in a content string and:
1. tokenizes the text
2. lowercases the token
3. removes stop words (commonly used words such as "a","an", "in")
4. keeps words with only alphabet characters (no punctuation)

In [114]:
import nltk
nltk.__version__

'3.2.5'

In [115]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
# Note that stopwords are all in lowercase format

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [120]:
# TODO: Create tokenized_content(content) function
def tokenized_content(content):
  words = nltk.word_tokenize(content)
  words = [word.lower() for word in words if word.isalpha()]
  words = [word for word in words if word not in stopwords]
  return words

Now, apply your `tokenize_content()` function to each piece of content in **top_content** and **regular_content** and flatten both of the lists to create **top_tokens** and **regular_tokens**


In [123]:
# TODO: tokenize and flatten
top_content = [tokenized_content(sent) for sent in top_content]
top_tokens = [word for sent in top_content for word in sent]
regular_content  = [tokenized_content(sent) for sent in regular_content]
regular_tokens = [word for sent in regular_content for word in sent]

In [125]:
# 1 point
grader.grade(test_case_id = 'test_top_tokens', answer = len(top_tokens))

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [124]:
# 1 point
grader.grade(test_case_id = 'test_regular_tokens', answer = len(regular_tokens))

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 3.3 Most Frequent Words
**TODO**: Now, find the 20 most common words amongst the content of `top_tokens` and `regular_tokens`. Return this as a list of `(word, count)` tuples, in descending order of `count`.

**Hint**: You can use `Counter` in this question: https://docs.python.org/2/library/collections.html#counter-objects

In [126]:
# TODO: Find 20 most common words amongst the content of the top and regular critic reviews
top_most_common = Counter(top_tokens).most_common(20)
regular_most_common = Counter(regular_tokens).most_common(20)

In [127]:
# 2 points
grader.grade(test_case_id = 'test_top_most_common', answer = top_most_common)

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [128]:
# 2 points
grader.grade(test_case_id = 'test_regular_most_common', answer = regular_most_common)

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


###3.4 Refining our Lists

Hmmm...both of these lists seem to display similar words. Let's try to tease out words that distinguish the high from the low scoring questions. 

One approach would be to find words in one list that are not in the other. This, however, may be too naive, as even if a word is extremely common in our high list, if it appears only once in our low list, it would get removed from consideration.

Let's instead find the difference between the counts within our two lists. Thus, if a word is really common in one, but not the other, the count would only decrease slightly. Alternatively, if a word is common in both lists, it would effectively zero out. 

However, given that the number of regular tokens is about three times that of top tokens, we need to make sure both lists are of equal length before applying the difference method. 

**TODO:** Randomly sample 1 million (1000000) tokens from `top_tokens` and `regular_tokens`, and place them in lists `top_sample` and `regular_sample` respectively.
-Then, using the difference method, create **distinct_top_counter** and **distinct_regular_counter** to find the top 20 counts of words within each group. Pass this into two lists, `distinct_top_most_common` and `distinct_regular_most_common` respectively. These should be lists of `(word, count)` tuples (`.most_common()` should help!)

Be careful on which list you are subtracting!


In [129]:
# TODO: randomly sample 1 million tokens from top_tokens and regular_tokens
top_sample = random.sample(top_tokens,1000000)
regular_sample = random.sample(regular_tokens,1000000)

In [141]:
# TODO: use difference to find the top 20 counts of words within each group
distinct_top = Counter(top_sample)
distinct_regular = Counter(regular_sample)
distinct_top.subtract(regular_sample)
distinct_regular.subtract(top_sample)

In [143]:
distinct_top_most_common = distinct_top.most_common(20)
distinct_regular_most_common = distinct_regular.most_common(20)

In [144]:
# 2 points
grader.grade(test_case_id = 'test_distinct_top_most_common', answer = distinct_top_most_common)

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


In [145]:
# 2 points
grader.grade(test_case_id = 'test_distinct_regular_most_common', answer = distinct_regular_most_common)

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 3.5 Word Clouds

Before we move on from this dataset, let's do one final step and visualize our results with wordclouds.

**TODO**: Take a look at [this documentation](https://amueller.github.io/word_cloud/generated/wordcloud.WordCloud.html) and create two word clouds for our two groups of distinct tokens.

Be sure to create these on the list of distinct tokens that you randomly sampled, and not just the top 20. We will be going through your notebooks and manually grading your world clouds (worth 4 points combined). 


In [ ]:
# TODO: make a word cloud for top tokens
from wordcloud import WordCloud
import matplotlib.pyplot as plt
top_plot = dict(distinct_top)
wc1 = WordCloud(width=2400,height=1500,).generate_from_frequencies(top_plot)
plt.imshow(wc1)
plt.axis("off")
plt.show()

In [ ]:
# TODO: make a word cloud for regular tokens
regular_plot = dict(distinct_regular)
wc2 = WordCloud(width=2400,height=1500,).generate_from_frequencies(regular_plot)
plt.imshow(wc2)
plt.axis("off")
plt.show()

# HW Submission

<br>
<center><img src = "https://memegenerator.net/img/instances/73124265/good-job.jpg" width= "500" align ="center"/></center>
<br>

Big congratulations for getting this far! The piece of good news is that similar to HW1, you basically know the score you have when you submit to Gradescope. However, this time, we will be manually grading your wordclouds, so the autograder score is not final! Remember also that since we'll be checking for plagiarism, make sure to cite your sources (if any) using simple urls / links.

Before you submit on Gradescope (you must submit your notebook to receive credit):

1.   Restart and Run-All to make sure there's nothing wrong with your notebook
2.   **Double check that you have the correct PennID (all numbers) in the autograder**. 
3. Make sure you've run all the PennGrader cells (and gotten the score that you want and deserve)
4. Go to the "File" tab at the top left, and click "Download .ipynb" + "Download .py" and upload both the Python file and ipnyb notebook to Gradescope directly, naming the files **"homework2.ipynb"** and **"homework2.py"** respectively!

###Be sure to name your files correctly!!!

**Let the course staff know ASAP if you have any issues submitting.**